In [3]:
# | echo: false
# | output: false

import sys
import os

sys.path.append(os.path.abspath("../../"))
sys.path.append(os.path.abspath("./social-media-handle-checker"))

%reload_ext autoreload
%autoreload 2

import asyncio
from typing import Any, Literal
from python_utils.get_browser import get_browser_page_async
from app import check_social_media_handle

## Instagram Account

In [7]:
await check_social_media_handle("instagram", "test")

{'available': True,
 'message': 'username <b>"test"</b> is ✅ Available. However, usernames from disabled or deleted accounts may also appear available but you can\'t choose them, eg: usernames like <b>"we"</b>, <b>"us"</b>, and <b>"the"</b>. Go to <a target="_blank" href="https://accountscenter.instagram.com/">accounts center</a> and try changing the username of an existing account and see if it it\'s available',
 'url': 'https://www.instagram.com/test/',
 'logs': [{'key': 'profile_response_username', 'value': ''},
  {'key': 'web_profile_response.status_code', 'value': 400},
  {'key': 'username_is_available_uri',
   'value': ('test', False, 'https://www.instagram.com/test/')}]}

In [2]:
tasks = [
    async_availability_status(resolve_instagram_username(username, lambda x, y: None)) 
    for username 
    in ['we', 'us', 'instagram']
]
await asyncio.gather(*tasks)

[{'available': True,
  'message': 'we: ✅ Available',
  'url': 'https://www.instagram.com/we/'},
 {'available': True,
  'message': 'us: ✅ Available',
  'url': 'https://www.instagram.com/us/'},
 {'available': False,
  'message': 'instagram: ❌ Taken',
  'url': 'https://www.instagram.com/instagram/'}]

## LinkedIn User Account

In [11]:
await async_availability_status(resolve_linkedin_username("USERNAME", "in"))

{'available': True,
 'message': 'USERNAME: ✅ Available',
 'url': 'https://www.linkedin.com/in/USERNAME'}

## LinkedIn Company Account

In [13]:
await async_availability_status(resolve_linkedin_username("USERNAME", "company"))

{'available': True,
 'message': 'USERNAME: ✅ Available',
 'url': 'https://www.linkedin.com/company/USERNAME'}

In [ ]:




def resolve_username_from_title(username: str):
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto(uri, wait_until='load')
        await page.wait_for_timeout(300)
        title: str = await page.title()
        await close()
        return username.lower() in title.lower()
    return resolve

def resolve_document_contains(search: str, evaluate_str: str) -> bool:
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto(uri)
        inner_text = await page.evaluate(evaluate_str)
        await close()
        return search.lower() in inner_text.lower()
    return resolve

class UsernameChecker:
    def __init__(self, username):
        self.username = username
        self.platforms: dict[Literal["name", "uri", "resolve", "message"], Any]= [
            # {
            #     "name": "LinkedIn Company Page",
            #     # can replace "www." with "de.", ".ke", ".ug", etc
            #     "uri": f"https://www.linkedin.com/company/{username}",
            #     "resolve": resolve_linkedin_username,
            # },
            # {
            #     "name": "LinkedIn User Account",
            #     # can replace "www." with "de.", ".ke", ".ug", etc
                  #     # # linkedin private user => kamau
            #     "uri": f"https://www.linkedin.com/in/{username}",
            #     "resolve": resolve_linkedin_username,
            #     # https://github.com/tomquirk/linkedin-api
            #     "message": (
            #         "❌ Taken or Private."
            #         " 😶 LinkedIn private user profiles are not easy to check,"
            #         " login into LinkedIn and go to"
            #         f' "https://www.linkedin.com/in/{username}"')
            # },
            {
                "name": "Instagram Account",
                "resolve": resolve_instagram_username(username),
            },
            # {
            #     "name": "Threads Account",
            #     # trailing / is important here
            #     "uri": f"https://www.instagram.com/{username}/",
            #     "resolve": resolve_instagram_username(username),
            # },
            # {
            #     "name": "Youtube Channel",
            #     "uri": f"https://www.youtube.com/@{username}",
            #     "resolve": resolve_document_contains(f"@{username}", "document.body.outerHTML"),
            # },
            # {
            #     "name": "Tiktok Account",
            #     "uri": f"https://www.tiktok.com/@{username}",
            #     "resolve": resolve_username_from_title(username),
            # },
            # {
            #     "name": "X/Twitter Account",
            #     "uri": f"https://x.com/{username}",
            #     # not accurate!
            #     # https://github.com/sherlock-project/sherlock/blob/master/sherlock_project/resources/data.json
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Pinterest Account",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Snapchat Account",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Facebook User Account",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Facebook Page",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Reddit User",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Reddit Subreddit",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Gmail Email",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # }
        ]

    async def check_async(self):
        tasks = [self._check_async(**platform) for platform in self.platforms]
        return await asyncio.gather(*tasks)




In [ ]:

# Create checker instance
checker = UsernameChecker("One_MorePlace")
# Check availability
await checker.check_async()